In [11]:
import requests
import pandas as pd
import requests
from datetime import datetime, timedelta

# Starting and ending dates
start_date = datetime.strptime('07.10.2023', '%d.%m.%Y')
end_date = datetime.now()

# List to store all the JSON data
all_data = []

# Iterating through each day from start to end
while start_date <= end_date:
    # Formatting date for the request URL
    formatted_date = start_date.strftime('%d.%m.%Y')
    alerts_url = f'https://www.oref.org.il//Shared/Ajax/GetAlarmsHistory.aspx?lang=he&fromDate={formatted_date}&toDate={formatted_date}&mode=0'
    
    # Making the request and appending the result
    alerts_json = requests.get(alerts_url).json()
    all_data.extend(alerts_json)  # Assuming the returned data is a list
    
    print('Done', formatted_date)
    
    # Moving to the next day
    start_date += timedelta(days=1)

# Break multi-region alerts into separate records
df = pd.DataFrame.from_records(all_data)
df['date'] = pd.to_datetime(df['date'])
df['data'] = df['data'].str.split(',')
df = df.explode('data')

# Remove sub-regions such as א, ב, ג, ד
df = df[df['data'].str.len() > 2]

# Change Hatzor to detailed name as the google geocoder fail to detect the correct city
df['data'] = df['data'].replace('חצור', 'חצור אשדוד')

# Show rocket sirens only
df = df[df['category_desc'] == 'ירי רקטות וטילים']

# Map city names to coordinates
def get_coordinates(city_name):
    city_name = city_name + ', ישראל'
    api_key = 'YOUR_API_KEY'
    geocoder_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={city_name}&key={api_key}&language=iw'
    geocoding_result = requests.get(geocoder_url).json()
    lat = geocoding_result['results'][0]['geometry']['location']['lat']
    long = geocoding_result['results'][0]['geometry']['location']['lng']
    return(lat, long)
    
city_to_coords = {}
for city in df['data'].unique():
    try:
        city_to_coords[city] = get_coordinates(city)
        print(city, '\t -', city_to_coords[city], end='\r')
    except Exception as e:
        city_to_coords[city] = (None, None)
        print(city, '\t', '- failed finding coordinates')

# Apply mapping on all data
df['outLat'] = df['data'].apply(lambda x: city_to_coords[x][0])
df['outLong'] = df['data'].apply(lambda x: city_to_coords[x][1])

# Define a function to determine the inLat and inLong based on the condition
def set_coordinates(row):
    if row['outLat'] > 32.8766248 and row['outLong'] > 35.2625196:  # north of the given point
        return 33.2677512, 35.5735599
    else:
        return 31.513, 34.452

df['inLat'], df['inLong'] = zip(*df.apply(set_coordinates, axis=1))

# Filter wrong coordinates outside of Israel polygon (only if you use a bad geocoder)
'''
filtered_df = df[(df['outLong'] < 35.8)
                & (df['outLong'] > 33.3)
                & (df['outLat'] < 34.0)
                & (df['outLat'] > 29.2)]

filtered_df.to_csv('RocketLaunchData - Filtered.csv', encoding='utf-8-sig', index=False)
display(filtered_df)
'''

df.to_csv('RocketLaunchData.csv', encoding='utf-8-sig', index=False)
display(df)

Done 07.10.2023
Done 08.10.2023
Done 09.10.2023
Done 10.10.2023
Done 11.10.2023
Done 12.10.2023
Done 13.10.2023
Done 14.10.2023
Done 15.10.2023
Done 16.10.2023
Done 17.10.2023
Done 18.10.2023
Done 19.10.2023
Done 20.10.2023
Done 21.10.2023
Done 22.10.2023
Done 23.10.2023
Done 24.10.2023
Done 25.10.2023
Done 26.10.2023
Done 27.10.2023
Done 28.10.2023
Done 29.10.2023
Done 30.10.2023
Done 31.10.2023
Done 01.11.2023
Done 02.11.2023


/var/folders/1c/n73yf83d03z0qkbr6wgxps500000gn/T/ipykernel_21161/3145207721.py:30: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['date'] = pd.to_datetime(df['date'])


סיט 	 - failed finding coordinates653)934528))75)000000001)


,data,date,time,alertDate,category,category_desc,matrix_id,rid,outLat,outLong,inLat,inLong
0,שדרות,2023-07-10,23:42:45,2023-10-07T23:43:00,1,ירי רקטות וטילים,1,20550,31.522694,34.595581,31.513,34.452
0,איבים,2023-07-10,23:42:45,2023-10-07T23:43:00,1,ירי רקטות וטילים,1,20550,31.533597,34.609697,31.513,34.452
0,ניר עם,2023-07-10,23:42:45,2023-10-07T23:43:00,1,ירי רקטות וטילים,1,20550,31.518459,34.580544,31.513,34.452
1,שוקדה,2023-07-10,23:13:53,2023-10-07T23:14:00,1,ירי רקטות וטילים,1,20549,31.422098,34.524383,31.513,34.452
2,בארי,2023-07-10,23:13:53,2023-10-07T23:14:00,1,ירי רקטות וטילים,1,20548,31.424513,34.492567,31.513,34.452
...,...,...,...,...,...,...,...,...,...,...,...,...
5739,אשדוד - ח,2023-02-11,09:50:30,2023-11-02T09:51:00,1,ירי רקטות וטילים,1,24283,31.804381,34.655314,31.513,34.452
5740,מבטחים,2023-02-11,06:40:03,2023-11-02T06:40:00,1,ירי רקטות וטילים,1,24282,31.242423,34.407488,31.513,34.452
5740,עמיעוז,2023-02-11,06:40:03,2023-11-02T06:40:00,1,ירי רקטות וטילים,1,24282,31.248463,34.413404,31.513,34.452
5740,ישע,2023-02-11,06:40:03,2023-11-02T06:40:00,1,ירי רקטות וטילים,1,24282,31.247595,34.403091,31.513,34.452


In [19]:
df = pd.DataFrame.from_records(all_data)
df['data'] = df['data'].str.split(',')
df = df.explode('data')

# Remove sub-regions such as א, ב, ג, ד
df = df[df['data'].str.len() > 2]

# Change Hatzor to detailed name as the google geocoder fail to detect the correct city
df['data'] = df['data'].replace('חצור', 'חצור אשדוד')

# Show rocket sirens only
df = df[df['category_desc'] == 'ירי רקטות וטילים']

df['outLat'] = df['data'].apply(lambda x: city_to_coords[x][0])
df['outLong'] = df['data'].apply(lambda x: city_to_coords[x][1])

# Define a function to determine the inLat and inLong based on the condition
def set_coordinates(row):
    if row['outLat'] > 32.8766248 and row['outLong'] > 35.2625196:  # north of the given point
        return 33.2677512, 35.5735599
    else:
        return 31.513, 34.452

df['inLat'], df['inLong'] = zip(*df.apply(set_coordinates, axis=1))
df.to_csv('RocketLaunchData.csv', encoding='utf-8-sig', index=False)

In [17]:
# Install instructions available at - https://docs.kepler.gl/docs/keplergl-jupyter#install
from keplergl import KeplerGl

config = {'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': ['OrefSirens'],
     'id': 'ed77mhj1',
     'name': ['datetime'],
     'type': 'timeRange',
     'value': [1659732960000, 1659739300000],
     'enlarged': True,
     'plotType': 'histogram',
     'animationWindow': 'free',
     'yAxis': None,
     'speed': 1}],
   'layers': [{'id': 'pd16gsv',
     'type': 'arc',
     'config': {'dataId': 'OrefSirens',
      'label': 'SirensEdges',
      'color': [255, 251, 152],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat0': 'inLat',
       'lng0': 'inLong',
       'lat1': 'outLat',
       'lng1': 'outLong'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'thickness': 0.5,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'sizeRange': [0, 10],
       'targetColor': [218, 0, 0]},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'id': 'p5giket',
     'type': 'heatmap',
     'config': {'dataId': 'OrefSirens',
      'label': 'SirensNodes',
      'color': [231, 159, 213],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'lat': 'outLat', 'lng': 'outLong'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10},
      'hidden': False,
      'textLabel': [{'field': None,
        'color': [255, 255, 255],
        'size': 18,
        'offset': [0, 0],
        'anchor': 'start',
        'alignment': 'center'}]},
     'visualChannels': {'weightField': None, 'weightScale': 'linear'}}],
   'interactionConfig': {'tooltip': {'fieldsToShow': {'OrefSirens': [{'name': 'data',
        'format': None},
       {'name': 'date', 'format': None},
       {'name': 'time', 'format': None},
       {'name': 'datetime', 'format': None},
       {'name': 'category', 'format': None}]},
     'compareMode': False,
     'compareType': 'absolute',
     'enabled': True},
    'brush': {'size': 0.5, 'enabled': False},
    'geocoder': {'enabled': False},
    'coordinate': {'enabled': False}},
   'layerBlending': 'normal',
   'splitMaps': [],
   'animationConfig': {'currentTime': None, 'speed': 1}},
  'mapState': {'bearing': -12.14678899082569,
   'dragRotate': True,
   'latitude': 31.36529666166856,
   'longitude': 34.579642984864655,
   'pitch': 52.32949284907476,
   'zoom': 9.2,
   'isSplit': False},
  'mapStyle': {'styleType': 'dark',
   'topLayerGroups': {},
   'visibleLayerGroups': {'label': True,
    'road': True,
    'border': True,
    'building': True,
    'water': True,
    'land': True,
    '3d building': False},
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'mapStyles': {}}}}

kepler_map = KeplerGl(config=config)
kepler_map.add_data(data=df, name='OrefSirens')

with open('kepler.html', 'wb') as file:
    file.write(kepler_map._repr_html_())

kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/yoavtepper/Library/Python/3.9/lib/python/site-packages/jupyter_client/session.py:719: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['OrefSirens'], 'id': 'ed77mh…